In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import requests
from my_tokenizer import CharDataset
import my_tests

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


#### Download dataset

In [ ]:
# Download the dataset if not present
data_path = Path('../data/shakespeare.txt')
data_path.parent.mkdir(parents=True, exist_ok=True)

if not data_path.exists():
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    text = requests.get(url).text
    with open(data_path, 'w') as f:
        f.write(text)
else:
    with open(data_path, 'r') as f:
        text = f.read()

dataset = CharDataset(text, block_size=128)
my_tests.test_tokenizer_roundtrip()

Original:  Hello, world!
Encoded:   [3, 5, 6, 6, 7, 2, 0, 9, 7, 8, 6, 4, 1]
Decoded:   Hello, world!
Vocab size: 10
Dataset length: 9

First sample:
Input:  tensor([3, 5, 6, 6])
Target: tensor([5, 6, 6, 7])
Input text:  'Hell'
Target text: 'ello'
Test passed!


#### Setting hyperparameters

In [3]:
# The project was tested with 12 layers, 8 attention heads, and 768 embedding dimensions, on a single GPU.
vocab_size = dataset.get_vocab_size()
embed_dim = 768
num_heads = 8
num_layers = 12